In [1]:
import csv
import re
import nltk
import random
import nltk
import pandas as pd
import pickle
import numpy as np
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Collection Data

**Offer a strategy and write code to either generate or collect enough samples for us to be
able to train a model extracting the recipient.**

I downloaded movie lines from [here](http://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html)
I created a conversation database and a name database. I also created a list of wake words, such as "tell" and "message". Then I randomly combine a wake word, a name, and one or two messages. See function command_gen() for detail.

**How many samples do you think we need? Why?**

Since the format/sentence structure doesn't have too many variation. I don't think we need a large amount of samples like general object recognition. 10-100 thousands sample should be enough.

**What limitations do you see to your approach?**
The wake word list might be limited. People might be use different words to ask the program to send messages. Samples often assume people have said perfect sentences. In reality, people might say something like "please send...ummm...Jack...wait no...John...a message"

## Create Message and Contact Databases

In [2]:
# Extract Lines and Names and Create a Contact Database and a Message Database
file = open("./cornell_movie-dialogs_corpus/movie_lines.txt", "rb") 
#file = open("./sample.txt", "rb")
contacts, messages = [], []
for line in file:
    line = line.decode('utf-8', errors='ignore').strip('\n')
    split_line = line.split("+++$+++")
    recipent = split_line[3].strip()
    sentences = nltk.sent_tokenize(split_line[-1].strip())
    #print(split_line)
    if len(recipent) < 1:
        continue
    else:
        contacts.append(recipent[0]+recipent[1:].lower())
    messages.extend(sentences)

# Remove Duplicate Contacts and Shuffle Messages
contacts = list(set(contacts))
random.shuffle(messages)

# Remove Names with Errors
new_contacts = []
for contact in contacts:
    if '\t\t\t' in contact:
        continue
    if ']' in contact:
        continue
    new_contacts.append(' '.join(contact.split()))
contacts = new_contacts    

# Create Contact Dict For Easy Lookup
contacts_dict = defaultdict(list)
for contact in contacts:
    contact_split = contact.split()
    contacts_dict[contact_split[0].lower()].append(contact)


In [3]:
wake_words = ['tell',
              'notify',
              'ask',
              'inform',
              'message',
              'text',
              'reply to',
              'mention to',
             ]

## Generate Commands

In [4]:
def command_gen(wake_words, messages, contacts):
    num_messages = len(messages)
    i = 0
    dataset = []
    recipents = []

    while i < num_messages:
        # Wake words
        wake_word = random.sample(wake_words, 1)[0]
        if random.random() < 0.5:
            command = "Can you "\
                      + wake_word\
                      + " "              
        else:
            command = wake_word[0].upper()\
                      + wake_word[1:]\
                      + " "

        # Recipent
        contact = random.sample(contacts, 1)[0]
        command += contact
        recipents.append(contact)

        # Message(s)
        if random.random() < 0.5:
            command += ' that'

        command += " " + messages[i]
        i += 1

        if random.random() < 0.25 and i < num_messages:
            command += " and that "\
                       + messages[i]
            i += 1

        dataset.append(command)
        
    return dataset, recipents

dataset, recipents = command_gen(wake_words, messages, contacts)

In [5]:
#pickle.dump(dataset, open('command_dataset.pickle', 'wb'))
#pickle.dump(recipents, open('recipents_dataset.pickle', 'wb'))

#dataset = pickle.load(open('command_dataset.pickle', 'rb'))
#recipents = pickle.load(open('recipents_dataset.pickle', 'rb'))

# First Model

**Discuss what the advantages and limitations of all these different approaches are.**
I chose two approaches, one is rule based and the other is logistic regression.
Rule based doesn't require many samples. But it could be high computational complexity. Because it searches a contact name thru the sentence. The complexity would be N x M. N is the number of words in the command, and M is the number of contacts a user has. The rule based approaches is implemented in funciton find_recipent_rule_base().

The second approach is a machine learning approach. I did a word tokenization, and turned each word into a data point. They are labeled as whether they are (or part of) a recipent or not. Advantatge is that it is robust to different sentence format. Also rather saying "Recipent not found in your contact list", it could say for example "John not found in your contact list". Limitations will be that it requires more data to achieve a certain level of accuracy.

**How did you select your features? Did you engineer new ones? Please describe in detail the entire process, including how you chose the training/validation/testing data split, how you did your hyper-parameter tuning, etc.**
For the logistic regression. I extracted the word before and the word after the data point. I also used the location in the sentence as a feature. My orgininal thought is to split the data into 80% training and 20% testing. Then within the 80% training, I would use cross-validation to tune the hyper-parameters. However at the end, the data is too large for my computer to handle. So I took 50000 data points from training set and 5000 data points from testing set. I still did a cross validation using the 50000 training data points and tuned the regularization of the logistic regression. Finally using the 5000 testing data points to evaluate the model


In [17]:
X_train, X_test, y_train, y_test = train_test_split(dataset, recipents, test_size=0.2)

## Rule Based

In [18]:
def find_recipent_rule_based(command):
    
    wake_word_said = False
    words = command.split()
    i = 0
    
    # only searching for recipent if wake word is said
    while i < len(words):
        if words[i].lower() in wake_words:
            i += 1
            wake_word_said = True
            break
        elif ' '.join(words[i:i+2]).lower() in wake_words:
            i += 2
            wake_word_said = True
            break
        i += 1
        
    # rearching for first word of recipent name
    if wake_word_said:
        while words[i].lower() not in contacts_dict and i < len(words):
            i += 1
            
    # when found, find the whole recipent name
    if i < len(words):
        if words[i].lower() in contacts_dict:
            sub_sentence = ' '.join(words[i:])
            max_matching_len = 0

            for name in contacts_dict[words[i].lower()]:
                if name in sub_sentence and len(name)>max_matching_len:
                    max_matching_len = len(name)
                    recipent = name
            try:
                return recipent
            except:
                print(contacts_dict[words[i].lower()])

        i += 1
    return None

In [20]:
y_pred_rb = []
for test in X_test:
    y_pred_rb.append(find_recipent_rule_based(test))

correct = 0
for y, y_bar in zip(y_test, y_pred_rb):
    correct += int(y==y_bar)
    if y != y_bar:
        print(y, y_bar)

rb_accuracy = correct/len(y_test)

## Machine Learning Approach - Logistic Regression

### Extract Features

In [9]:
def extract_feature_label(command, label, n):

    words = nltk.word_tokenize(command)
    words=[word.lower() for word in words if word.isalpha()]
    features = []
    labels = []

    num_words = len(words)
    for i, word in enumerate(words):
        feature = []
        for j in range(n, 0, -1):
            k = i-j
            if k >= 0:
                feature.append(words[k])
            else:
                feature.append('NA')
        for j in range(1, n+1):
            k = i+j
            if k < num_words:
                feature.append(words[k])
            else:
                feature.append('NA')
        feature.append(i)
        features.append(feature)

        if word in label.lower().split():
            labels.append(1)
        else:
            labels.append(0)
    return features, labels

def sentence_to_words(X, y, n_neighbor=1, balanced=True):

    X_word_token, y_word_token = [], []
    for command, recipent in zip(X, y):
        features, labels = extract_feature_label(command, recipent, n_neighbor)
        X_word_token += features
        y_word_token += labels

    if balanced:
        ratio = sum(y_word_token)/len(y_word_token)

        X_word_token_blc, y_word_token_blc = [], []
        for feature, label in zip(X_word_token, y_word_token):
            if label == 0 and random.random() > ratio:
                continue
            X_word_token_blc.append(feature)
            y_word_token_blc.append(label)
        
        X_word_token = X_word_token_blc
        y_word_token = y_word_token_blc
    dataset = np.concatenate((np.array(X_word_token), 
                              np.array(y_word_token).reshape(-1,1)), 
                              axis=1)


    fwd, back = [], []
    for i in range(n_neighbor):
        fwd.append('-'+str(i+1)+'_loc')
        back.append('+'+str(i+1)+'_loc')
    fwd.reverse()
    col_names = fwd+back+['loc','label']
    
    return pd.DataFrame(dataset, columns=col_names)

df = sentence_to_words(dataset, recipents)

In [10]:
#pickle.dump(df, open('df.pickle', 'wb'))
#df = pickle.load(open('df.pickle', 'rb'))

In [11]:
# Take a sample due to large size of original dataset
train_df, test_df = train_test_split(df)
sample_train_df = train_df.sample(50000)
sample_test_df = test_df.sample(10000)

### CountVectorizer - Extract Numerical Values from Texts

In [12]:
stop_words = ENGLISH_STOP_WORDS.union('NA')

class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]    

CV_fwd = CountVectorizer(stop_words=stop_words, tokenizer=LemmaTokenizer())
CV_bck = CountVectorizer(stop_words=stop_words, tokenizer=LemmaTokenizer())
CV_fwd.fit(sample_train_df['-1_loc'])
CV_bck.fit(sample_train_df['+1_loc'])
    
def extract_feature(df):
    X_fwd = CV_fwd.transform(df['-1_loc'])
    X_bck = CV_bck.transform(df['+1_loc'])

    loc = np.array(df['loc'].tolist()).reshape(-1,1).astype(int)
    X = np.concatenate((X_fwd.toarray(), X_bck.toarray(), loc), axis=1)

    y = np.array(df['label'].tolist()).astype(int)
    
    return X, y

X_train, y_train = extract_feature(sample_train_df)
X_test, y_test = extract_feature(sample_test_df)

### Logistic Regression (With GridSearchCV to Tune Hyper-Parameters)

In [13]:
LR = LogisticRegression()
LR.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [14]:
parameters = {'C':[0.1, 1, 10], 'penalty':['l1','l2']}
LR = LogisticRegression()
clf = GridSearchCV(LR, parameters, cv=5, n_jobs=-1, verbose=True)
clf.fit(X_train, y_train)


LR = clf.best_estimator_

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  3.4min finished


0.9243

# Model Evaluation 

**What is the accuracy for each one of your models? How much data did you use to evaluate your model? Is it better/worse than you expected? Is the model overfitting? If so, how do you ‘fix’ it? **

Rule based achieved 100%, but I wouldn't expect rule based approach did very good in the real world scenarios. I created those sample data therefore I know very well how to find the recipents. It is almost like cheating.
The Logistic Regression achieved a 92.43% on the testing sample data. It is actually better than I expected since I only used one word before and one word after the word I am classifying. However, again, the data I generated might not be representative enough. More brainstorming of how to collect/generate data is needed.

**Then think of other metrics you’d like to look at to evaluate the ‘goodness’ of your model. Why do you think these metrics are important?**
Computational complexity is an important metrics since customers wouldn't like to wait forever to send a message.

## Rule Based

In [21]:
rb_accuracy

1.0

## Logistic Regression

In [15]:
LR.score(X_test, y_test)

0.9243

# A More Sophisticated Model

I didn't have enough time to finish this. I did look into (Stanford's CRF-NER)[https://nlp.stanford.edu/software/CRF-NER.shtml] but I didn't have the expertise to complete a sophisticated model in a week. I also thought of using Spacy to identify part-of-speech of each token. Instead extract the word before and after the one we're trying to classify (like I did for the logistic regression), we can extract the POS instead. This approach could be more robust and reduce feature dimenstion by a lot.

In [22]:
import spacy

nlp = spacy.load('en')
doc = nlp(dataset[5])

for token in doc:
    print(token.text, token.pos_, token.tag_)

Can VERB MD
you PRON PRP
reply VERB VB
to ADP IN
Tricks NOUN NNS
that ADJ WDT
Come VERB VBP
here ADV RB
, PUNCT ,
Brenna PROPN NNP
. PUNCT .
